In [86]:
#importing modules
import pandas as pd
import requests
import pickle
import string, os, sys, json, math, random
from textblob import TextBlob, Word

In [87]:
#creating dataframe
df = pd.read_csv('train.csv')

In [88]:
df.head()

,UID,comment,date,parent_comment,score
0,Tr-1,NC and NH.,2016-10,"Yeah, I get that argument. At this point, I'd ...",2
1,Tr-2,You do know west teams play against west teams...,2016-11,The blazers and Mavericks (The wests 5 and 6 s...,-4
2,Tr-3,"They were underdogs earlier today, but since G...",2016-09,They're favored to win.,3
3,Tr-4,"This meme isn't funny none of the ""new york ni...",2016-10,deadass don't kill my buzz,-8
4,Tr-5,I could use one of those tools.,2016-12,Yep can confirm I saw the tool they use for th...,6


In [89]:
df["comments"] = df['parent_comment'].astype(str) + df['comment']

In [90]:
df['comments'][0]

"Yeah, I get that argument. At this point, I'd prefer is she lived in NC as well.NC and NH."

In [91]:
del df["comment"]

In [92]:
df.drop(['UID','parent_comment','date'],axis=1,inplace=True)

In [93]:
df.head()

,score,comments
0,2,"Yeah, I get that argument. At this point, I'd ..."
1,-4,The blazers and Mavericks (The wests 5 and 6 s...
2,3,They're favored to win.They were underdogs ear...
3,-8,deadass don't kill my buzzThis meme isn't funn...
4,6,Yep can confirm I saw the tool they use for th...


In [94]:
df = df[['comments','score']]

In [95]:
df.head()

,comments,score
0,"Yeah, I get that argument. At this point, I'd ...",2
1,The blazers and Mavericks (The wests 5 and 6 s...,-4
2,They're favored to win.They were underdogs ear...,3
3,deadass don't kill my buzzThis meme isn't funn...,-8
4,Yep can confirm I saw the tool they use for th...,6


In [96]:
#dimension of dataset
df.shape

(45000, 2)

In [97]:
df['score'].value_counts()

 1       15430
 2        7250
 3        4078
 0        1923
 4        1892
 5        1832
 6        1371
 7        1070
 8         830
-1         746
 9         661
 10        569
 11        514
 12        388
-2         377
 13        377
 14        324
 15        287
 16        240
 17        233
-3         215
 18        203
 19        194
 20        170
-4         164
 22        150
 21        144
-5         127
 23        116
 24        115
         ...  
 211         1
 179         1
 1746        1
 626         1
 562         1
 338         1
 274         1
 210         1
 146         1
-142         1
-46          1
 561         1
 401         1
 241         1
 144         1
 209         1
 177         1
 145         1
-47          1
 912         1
 784         1
 720         1
 592         1
 528         1
 496         1
 432         1
 304         1
 208         1
 176         1
 1039        1
Name: score, Length: 427, dtype: int64

In [98]:
# Importing NLTK Library

import re
import nltk
from nltk.corpus import stopwords

In [99]:
# Dataset Cleaning - 1 

def processComment(cmt):
    # process the comments

    #Convert to lower case
    cmt = cmt.lower()
    #Convert www.* or https?://* to URL
    cmt = re.sub('((www\.[^\s]+)|(https?://[^\s]+))','URL',cmt)
    #Convert @username to AT_USER
    cmt = re.sub('@[^\s]+','AT_USER',cmt)
    #Remove additional white spaces
    cmt = re.sub('[\s]+', ' ', cmt)
    #Replace #word with word
    cmt = re.sub(r'#([^\s]+)', r'\1', cmt)
    #trim
    
    cmt = cmt.strip('\'"')
    return cmt

In [100]:
# Dataset Cleaning - 2

#start replaceTwoOrMore
def replaceTwoOrMore(s):
    #look for 2 or more repetitions of character and replace with the character itself
    pattern = re.compile(r"(.)\1{1,}", re.DOTALL)
    return pattern.sub(r"\1\1", s)
#end

In [101]:
# Dataset Cleaning - 3
# Removing Stop Words from Dataset

def getStopWordList(stopWordListFileName):
    #read the stopwords file and build a list
    stopWords = []

    fp = open(stopWordListFileName, 'r')
    line = fp.readline()
    while line:
        word = line.strip()
        stopWords.append(word)
        line = fp.readline()
    fp.close()
    return stopWords

In [102]:
# Extracting features

def getFeatureVector(cmt):
    featureVector = []
    #split comments into words
    words = cmt.split()
    for w in words:
        #replace two or more with two occurrences
        w = replaceTwoOrMore(w)
        #strip punctuation
        w = w.strip('\'"?,.')
        #check if the word stats with an alphabet
        val = re.search(r"^[a-zA-Z][a-zA-Z0-9]*$", w)
        #ignore if it is a stop word
        if(w in stopWords or val is None):
            continue
        else:
            featureVector.append(w.lower())
    return featureVector
#end

In [103]:
# stop words file

st = open('stopwords.txt', 'r')


stopWords = getStopWordList('stopwords.txt')

In [104]:
cmt_list = []

In [105]:
# Data-Preprocessing-Dataset
for cmt in df['comments']:
    cm = processComment(cmt)
    cm1 = getFeatureVector(cm)
    cmt_list.append(cm1)

In [106]:
cmt_list[4]

['yep', 'confirm', 'tool', 'boy', 'tools']

In [107]:
labels=[]
for l in df['score']:
    labels.append(l)

In [108]:
comments = cmt_list

In [109]:
comments[8]

['upgrade',
 'spend',
 'birthday',
 'upgrade',
 'spend',
 'spend',
 'little',
 'specs',
 'cm',
 'evo',
 'corsair',
 'vengence',
 'lpx',
 'evga',
 'gtx',
 'sc',
 'corsair',
 'spec',
 'alpha',
 'evga',
 'p2',
 'gb',
 'corsair',
 'force',
 'le',
 'benq',
 'xl2411z',
 'razer',
 'blackwidow',
 'tournament',
 'edition',
 'thinking',
 'wd',
 'blue',
 'ram',
 'razer',
 'blackwidow',
 'keyboard',
 'actual',
 'cherry',
 'mx',
 'switches',
 'offer',
 'features',
 'cable',
 'volta',
 'cardyou',
 'build',
 'buy',
 'games',
 'save']

In [110]:
labels[8]

1

In [111]:
x2 = [" ".join(c) for c in cmt_list]

In [112]:
df2 = pd.DataFrame({'comments': x2, 'label':labels})

In [113]:
df2.head()

,comments,label
0,yeah argument prefer lived nc nh,2
1,blazers mavericks wests carry record playoffs ...,-4
2,favored underdogs earlier announcement afterno...,3
3,deadass kill buzzthis meme funny none york nig...,-8
4,yep confirm tool boy tools,6


In [114]:
from sklearn.utils import shuffle
df2 = shuffle(df2)

In [115]:
df2['comments'].fillna(value='Missing', inplace=True)

In [116]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.metrics import mean_squared_log_error
from sklearn.linear_model import SGDRegressor
from sklearn.pipeline import Pipeline

In [117]:
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.naive_bayes import MultinomialNB
X_train, X_test, y_train, y_test = train_test_split(df2['comments'], df2['label'], random_state = 0)
count_vect = CountVectorizer()
X_train_counts = count_vect.fit_transform(X_train)
tfidf_transformer = TfidfTransformer()
X_train_tfidf = tfidf_transformer.fit_transform(X_train_counts)

In [118]:
X_train_tfidf.shape

(33750, 49626)

In [119]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.metrics import mean_squared_log_error
from sklearn.linear_model import SGDRegressor
from sklearn.pipeline import Pipeline
seed=101
import time

In [120]:
#preparing model
model = SGDRegressor(loss='squared_loss', penalty='l2', random_state=seed, max_iter=5)
params = {'penalty':['none','l2','l1'],
          'alpha':[1e-4, 2e-4, 5e-4, 1e-3, 2e-3, 5e-3, 1e-2, 2e-2, 5e-2, 0.1]}
gs = GridSearchCV(estimator=model,
                  param_grid=params,
                  scoring='neg_mean_squared_error',
                  n_jobs=1,
                  cv=5,
                  verbose=3)
start = time.time()
gs.fit(X_train_tfidf,y_train)
end = time.time()
print('Time to train model: %0.2fs' % (end -start))

Fitting 5 folds for each of 30 candidates, totalling 150 fits
[CV] alpha=0.0001, penalty=none ......................................
[CV]  alpha=0.0001, penalty=none, score=-1589.9077087717544, total=   0.1s
[CV] alpha=0.0001, penalty=none ......................................
[CV]  alpha=0.0001, penalty=none, score=-3232.8402814195165, total=   0.0s
[CV] alpha=0.0001, penalty=none ......................................
[CV]  alpha=0.0001, penalty=none, score=-3889.090143265671, total=   0.0s
[CV] alpha=0.0001, penalty=none ......................................
[CV]  alpha=0.0001, penalty=none, score=-1595.0201106855047, total=   0.0s
[CV] alpha=0.0001, penalty=none ......................................


[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.1s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   2 out of   2 | elapsed:    0.1s remaining:    0.0s


[CV]  alpha=0.0001, penalty=none, score=-1327.5132338281517, total=   0.0s
[CV] alpha=0.0001, penalty=l2 ........................................
[CV]  alpha=0.0001, penalty=l2, score=-1589.91755426569, total=   0.0s
[CV] alpha=0.0001, penalty=l2 ........................................
[CV]  alpha=0.0001, penalty=l2, score=-3232.858741588111, total=   0.0s
[CV] alpha=0.0001, penalty=l2 ........................................
[CV]  alpha=0.0001, penalty=l2, score=-3889.105762951288, total=   0.0s
[CV] alpha=0.0001, penalty=l2 ........................................
[CV]  alpha=0.0001, penalty=l2, score=-1595.0445199705207, total=   0.0s
[CV] alpha=0.0001, penalty=l2 ........................................
[CV]  alpha=0.0001, penalty=l2, score=-1327.5214038359843, total=   0.0s
[CV] alpha=0.0001, penalty=l1 ........................................
[CV]  alpha=0.0001, penalty=l1, score=-1589.9371592194884, total=   0.1s
[CV] alpha=0.0001, penalty=l1 ...................................

[CV]  alpha=0.002, penalty=none, score=-1327.5132338281517, total=   0.1s
[CV] alpha=0.002, penalty=l2 .........................................
[CV]  alpha=0.002, penalty=l2, score=-1590.1015287244722, total=   0.1s
[CV] alpha=0.002, penalty=l2 .........................................
[CV]  alpha=0.002, penalty=l2, score=-3233.195533796311, total=   0.1s
[CV] alpha=0.002, penalty=l2 .........................................
[CV]  alpha=0.002, penalty=l2, score=-3889.391866032726, total=   0.1s
[CV] alpha=0.002, penalty=l2 .........................................
[CV]  alpha=0.002, penalty=l2, score=-1595.486774795345, total=   0.0s
[CV] alpha=0.002, penalty=l2 .........................................
[CV]  alpha=0.002, penalty=l2, score=-1327.6757183814261, total=   0.1s
[CV] alpha=0.002, penalty=l1 .........................................
[CV]  alpha=0.002, penalty=l1, score=-1590.4240881655578, total=   0.1s
[CV] alpha=0.002, penalty=l1 .........................................


[CV]  alpha=0.05, penalty=none, score=-1595.0201106855047, total=   0.0s
[CV] alpha=0.05, penalty=none ........................................
[CV]  alpha=0.05, penalty=none, score=-1327.5132338281517, total=   0.0s
[CV] alpha=0.05, penalty=l2 ..........................................
[CV]  alpha=0.05, penalty=l2, score=-1591.9716794259543, total=   0.0s
[CV] alpha=0.05, penalty=l2 ..........................................
[CV]  alpha=0.05, penalty=l2, score=-3236.2267010868118, total=   0.0s
[CV] alpha=0.05, penalty=l2 ..........................................
[CV]  alpha=0.05, penalty=l2, score=-3891.9979337295163, total=   0.0s
[CV] alpha=0.05, penalty=l2 ..........................................
[CV]  alpha=0.05, penalty=l2, score=-1599.3022924278398, total=   0.0s
[CV] alpha=0.05, penalty=l2 ..........................................
[CV]  alpha=0.05, penalty=l2, score=-1329.3195372128382, total=   0.0s
[CV] alpha=0.05, penalty=l1 ..........................................
[C

[Parallel(n_jobs=1)]: Done 150 out of 150 | elapsed:    7.8s finished


In [121]:
model = gs.best_estimator_
print(gs.best_params_)
print(gs.best_score_)

{'alpha': 0.0001, 'penalty': 'none'}
-2326.87429559


In [122]:
import numpy as np

In [123]:
X_test_counts = count_vect.transform(X_test)
X_test_tfidf = tfidf_transformer.transform(X_test_counts)

In [124]:
pipe = Pipeline([('vect',tfidf_transformer),('model',model)])
start = time.time()
y_pred = pipe.predict(X_test_tfidf)
end = time.time()
print('Time to generate predictions on test set: %0.2fs' % (end - start))

Time to generate predictions on test set: 0.01s


In [125]:
from sklearn import metrics

In [126]:
print(np.sqrt(metrics.mean_squared_error(y_test,y_pred)))

47.0276819699


In [127]:
# reading test file

In [128]:
df_test = pd.read_csv('test.csv')

In [129]:
df_test.head()

,UID,comment,date,parent_comment
0,Te-1,"Brb, gonna farm my homu",2016-10,So now I KNOW I've sold my soul to the Devil...
1,Te-2,TIL Russia is in Europe,2016-10,The entire CONTINENT of Europe? But arent they...
2,Te-3,Glad they filmed this with an 8mm.,2016-10,Girl drinks whole beer without using her hands
3,Te-4,But what about that cinematic feel?,2016-10,MRW someone defends 24 FPS over 60
4,Te-5,32 spots left as of today.,2016-10,Are there still spots open for this?


In [130]:
df_test["comments"] = df_test['parent_comment'].astype(str) + df_test['comment']

In [131]:
del df_test["comment"]

In [132]:
df_test.drop(['parent_comment','date'],axis=1,inplace=True)

In [133]:
df_test.head()

,UID,comments
0,Te-1,So now I KNOW I've sold my soul to the Devil.....
1,Te-2,The entire CONTINENT of Europe? But arent they...
2,Te-3,Girl drinks whole beer without using her hands...
3,Te-4,MRW someone defends 24 FPS over 60But what abo...
4,Te-5,Are there still spots open for this?32 spots l...


In [134]:
cmt_test_list = []
df_test['comments'].fillna('Missing', inplace=True)

In [135]:
for cmt in df_test['comments']:
    cm = processComment(cmt)
    cm1 = getFeatureVector(cm)
    cmt_test_list.append(cm1)

In [136]:
x3 = [" ".join(c) for c in cmt_test_list]

In [137]:
df_test = pd.DataFrame({'comments': x3})

In [138]:
df_test.head()

,comments
0,sold soul gonna farm homu
1,entire continent europe arent russia europe
2,girl drinks beer using handsglad filmed
3,mrw defends fps cinematic feel
4,spots spots left


In [139]:
X_test_counts2 = count_vect.transform(df_test['comments'].values)

In [140]:
X_test_tfidf2 = tfidf_transformer.transform(X_test_counts2)

In [141]:
df_test['score'] = gs.predict(count_vect.transform(df_test['comments'].values))

In [142]:
df_test.head()

,comments,score
0,sold soul gonna farm homu,7.374138
1,entire continent europe arent russia europe,8.726652
2,girl drinks beer using handsglad filmed,7.418216
3,mrw defends fps cinematic feel,6.474942
4,spots spots left,6.031600


In [143]:
df_final = pd.read_csv('test.csv')

In [144]:
df_final.drop(['comment','date','parent_comment'],axis=1,inplace=True)

In [145]:
df_final['score'] = gs.predict(count_vect.transform(df_test['comments'].values))

In [146]:
df_final.head()

,UID,score
0,Te-1,7.374138
1,Te-2,8.726652
2,Te-3,7.418216
3,Te-4,6.474942
4,Te-5,6.031600


In [150]:
df_final.to_csv('output.csv', index=False)

In [151]:
from sklearn import metrics

In [152]:
#RMSE
print(np.sqrt(metrics.mean_squared_error(y_test,y_pred)))

47.0276819699
